In [4]:
import tensorflow as tf
import numpy as np

# Define the paths to the text files containing the file paths
train_data_path = "./custom-speech/training_list.txt"
val_files_path = "./custom-speech/validation_list.txt"
test_files_path = "./custom-speech/testing_list.txt"

# Define the function to load the audio data and labels from text files
import os

def load_data(files_path):
    with open(files_path) as f:
        file_paths = f.read().splitlines()

    data = []
    labels = []
    missing_files = [] # to store the missing file paths
    for file_path in file_paths:
        # Load the audio signal
        try:
            audio_signal, _ = tf.audio.decode_wav(tf.io.read_file(file_path))
            audio_signal = audio_signal.numpy().flatten()
        except:
            missing_files.append(file_path)
            continue

        # Get the label from the directory name
        label = os.path.basename(os.path.dirname(file_path))

        # Append the audio signal and label to the data and labels lists
        data.append(audio_signal)
        labels.append(label)

    if missing_files: # log the missing file paths
        print(f"Skipped {len(missing_files)} files:")
        for path in missing_files:
            print(path)

    return np.array(data), np.array(labels)


# Load the training data and labels
train_data, train_labels = load_data(train_data_path)

# Load the validation data and labels
val_data, val_labels = load_data(val_files_path)

# Load the test data and labels
test_data, test_labels = load_data(test_files_path)

# Load the model from the saved file
model = tf.keras.models.load_model("./models/model.h5")

# Define the function to calculate accuracy
def calculate_accuracy(data, labels):
    # Convert the labels to one-hot encoding
    label_encoder = tf.keras.preprocessing.text.Tokenizer()
    label_encoder.fit_on_texts(labels)
    labels_encoded = label_encoder.texts_to_matrix(labels)

    # Evaluate the model on the data
    loss, accuracy = model.evaluate(data, labels_encoded)

    return accuracy

# Calculate the accuracy on the training data
train_accuracy = calculate_accuracy(train_data, train_labels)
print("Training accuracy:", train_accuracy)

# Calculate the accuracy on the validation data
val_accuracy = calculate_accuracy(val_data, val_labels)
print("Validation accuracy:", val_accuracy)

# Calculate the accuracy on the test data
test_accuracy = calculate_accuracy(test_data, test_labels)
print("Test accuracy:", test_accuracy)

def calculate_frr(data_path, class_label):
    with open(data_path) as f:
        file_paths = f.read().splitlines()

    # Convert the labels to one-hot encoding
    label_encoder = tf.keras.preprocessing.text.Tokenizer()
    label_encoder.fit_on_texts([class_label])
    class_index = np.argmax(label_encoder.texts_to_matrix([class_label]))

    # Load the audio data and labels
    data = []
    labels = []
    for file_path in file_paths:
        # Load the audio signal
        audio_signal, _ = tf.audio.decode_wav(tf.io.read_file(file_path))
        audio_signal = audio_signal.numpy().flatten()

        # Get the label from the directory name
        label = os.path.basename(os.path.dirname(file_path))

        # Append the audio signal and label to the data and labels lists
        data.append(audio_signal)
        labels.append(label)

    # Convert the labels to one-hot encoding
    labels_encoded = label_encoder.texts_to_matrix(labels)

    # Get the predictions from the model
    predictions = model.predict(np.array(data))

    # Get the predicted labels for the given class
    predicted_labels = np.argmax(predictions, axis=1)
    predicted_class = np.where(predicted_labels == class_index)[0]

    # Calculate the false rejection rate
    frr = 1 - (len(predicted_class) / len(labels))

    return frr

    
# Print the number of parameters and MACs in the model
print("Number of parameters:", model.count_params())
print("Number of MACs:", tf.profiler.experimental.profile(model, tf.profiler.experimental.ProfileOptionBuilder(
            tf.profiler.experimental.ProfileOptionBuilder.float_operation())
    .with_node_names(show_name_regexes=['.*'])
    .build()).total_float_ops)

# Print the input tensor shape and sampling rate
print("Input tensor shape:", model.input_shape)
print("Sampling rate:", 16000)

# Calculate the false alarm rate per hour
fa_rate = test_data.shape[0] * (1 - test_accuracy) / (len(test_labels) / 16000) * 3600
print("False Alarm Rate (FA/hour):", fa_rate)

2023-05-04 21:05:05.394980: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/b83c1acf_nohash_2.wav; No such file or directory
2023-05-04 21:05:05.395453: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: computer/computer_Pitch_Shift_6_324.wav; No such file or directory
2023-05-04 21:05:05.395676: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: yes/856eb138_nohash_4.wav; No such file or directory
2023-05-04 21:05:05.395895: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/57152045_nohash_0.wav; No such file or directory
2023-05-04 21:05:05.396106: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: yes/2cec1d47_nohash_0.wav; No such file or directory
2023-05-04 21:05:05.396300: W tens

2023-05-04 21:05:05.594632: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: computer/computer_Pitch_Shift_7_1764.wav; No such file or directory
2023-05-04 21:05:05.594669: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/ce9410da_nohash_0.wav; No such file or directory
2023-05-04 21:05:05.594706: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: computer/computer_Pitch_Shift_7_1193.wav; No such file or directory
2023-05-04 21:05:05.594744: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: computer/computer_Pitch_Shift_7_884.wav; No such file or directory
2023-05-04 21:05:05.594781: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: computer/computer_Pitch_Shift_5_28.wav; No such file or di

2023-05-04 21:05:05.802159: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: computer/computer_Pitch_Shift_7_255.wav; No such file or directory
2023-05-04 21:05:05.802356: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/0474c92a_nohash_0.wav; No such file or directory
2023-05-04 21:05:05.802526: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: yes/f5341341_nohash_1.wav; No such file or directory
2023-05-04 21:05:05.802607: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/7c83c5f6_nohash_0.wav; No such file or directory
2023-05-04 21:05:05.802720: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/efbc3952_nohash_0.wav; No such file or directory
2023-05-04 21:05:05.802768: W ten

Skipped 8291 files:
left/b83c1acf_nohash_2.wav
computer/computer_Pitch_Shift_6_324.wav
yes/856eb138_nohash_4.wav
left/57152045_nohash_0.wav
yes/2cec1d47_nohash_0.wav
computer/computer_Pitch_Shift_6_180.wav
left/978240e1_nohash_2.wav
computer/computer_Pitch_Shift_7_1234.wav
left/39999a0f_nohash_0.wav
left/c71e3acc_nohash_0.wav
yes/723efc4c_nohash_0.wav
computer/computer_Pitch_Shift_6_213.wav
computer/computer_Pitch_Shift_6_373.wav
computer/computer_Pitch_Shift_7_1713.wav
computer/computer_Pitch_Shift_7_732.wav
yes/3b3d2f59_nohash_3.wav
computer/computer_Pitch_Shift_5_77.wav
yes/ccca5655_nohash_1.wav
computer/computer_Pitch_Shift_6_64.wav
computer/computer_Pitch_Shift_6_545.wav
computer/computer_Pitch_Shift_7_152.wav
yes/eeaf97c3_nohash_3.wav
yes/a05a90c1_nohash_1.wav
computer/computer_Pitch_Shift_2_6.wav
computer/computer_Pitch_Shift_5_26.wav
yes/6f1ffef6_nohash_0.wav
yes/ec989d6d_nohash_2.wav
computer/computer_Pitch_Shift_7_1742.wav
yes/b00c4c53_nohash_1.wav
computer/computer_Pitch_Shi

2023-05-04 21:05:06.003689: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: computer/computer_Pitch_Shift_7_394.wav; No such file or directory
2023-05-04 21:05:06.006470: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/c41d9052_nohash_0.wav; No such file or directory
2023-05-04 21:05:06.006524: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/197f4153_nohash_2.wav; No such file or directory
2023-05-04 21:05:06.006567: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: left/bdb90d38_nohash_0.wav; No such file or directory
2023-05-04 21:05:06.006612: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: yes/aa233654_nohash_0.wav; No such file or directory
2023-05-04 21:05:06.006654: W ten

ValueError: Specify a dimension (`num_words` argument), or fit on some text data first.